In [1]:
#Installation
#!pip install requests --upgrade #upgrades system
import requests #We can use requests.get to download a page
import re
import pandas as pd
import string
import math
import random
#We can use the BeautifulSoup class to parse an HTML document.
#!pip install beautifulsoup4
from bs4 import BeautifulSoup

In [2]:
#Here we need to define headers in this function because google scholar webpage requires a login
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
url = 'https://scholar.google.com/scholar?cites=11660772839861173263&as_sdt=2005&sciodt=0,5&hl=en'
response =requests.get(url,headers=headers)


In [3]:
response.status_code # checks to make sure response is going through, should return 200

200

In [4]:
soup = BeautifulSoup(response.content,'html.parser') #sets up parser object

In [5]:
# this function for the getting inforamtion of the web page
def get_paperinfo(paper_url):

  #download the page
  response =requests.get(url,headers=headers)

  # check successful response
  if response.status_code != 200:
    print('Status code:', response.status_code)
    raise Exception('Failed to fetch web page ')

  #parse using beautiful soup
  paper_doc = BeautifulSoup(response.text,'html.parser')

  return paper_doc

In [6]:
paper = get_paperinfo(url) #getting paper info

In [7]:
#Scraping Functions
# this function for the extracting information of the tags
def get_tags(doc):
  paper_tag = doc.select('[data-lid]')
  cite_tag = doc.select('[name="Cites"] + a')
  link_tag = doc.find_all('h3',{"class" : "gs_rt"})
  author_tag = doc.find_all("div", {"class": "gs_a"})

  return paper_tag,cite_tag,link_tag,author_tag

# it will return the title of the paper
def get_papertitle(paper_tag):
  
  paper_names = []
  
  for tag in paper_tag:
    paper_names.append(tag.select('h3')[0].get_text())

  return paper_names

# function for the getting link information
def get_link(link_tag):

  links = []

  for i in range(len(link_tag)) :
    links.append(link_tag[i].a['href']) 

  return links 

# it will return the number of citation of the paper
def get_citecount(cite_tag):
  cite_count = []
  for i in cite_tag:
    cite = i.text
    if i is None or cite is None:  # if paper has no citatation then consider 0
      cite_count.append(0)
    else:
      tmp = re.search(r'\d+', cite) # its handle the None type object error and re use to remove the string " cited by " and return only integer value
      if tmp is None :
        cite_count.append(0)
      else :
        cite_count.append(int(tmp.group()))

  return cite_count

# function for the getting autho , year and publication information
def get_author_year_publi_info(authors_tag):
  years = []
  publication = []
  authors = []
  for i in range(len(authors_tag)):
      authortag_text = (authors_tag[i].text).split()
      year = int(re.search(r'\d+', authors_tag[i].text).group())
      years.append(year)
      publication.append(authortag_text[-1])
      author = authortag_text[0] + ' ' + re.sub(',','', authortag_text[1])
      authors.append(author)
      #sleep(3)
  
  return years , publication, authors



In [10]:
#get num of results, to iterate through all the pages
def get_page_num(doc, incr):
    result = doc.find_all( "div" , {"class": "gs_ab_mdw"})
    val = result[1].get_text()
    num = int(re.search(r'\d+', val).group())
    page = int(incr*math.ceil(num/incr))
    return page

In [9]:
# creating final repository
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                    #'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

# adding information in repository

def add_in_paper_repo(papername,year,author,publi,link):
    paper_repos_dict['Paper Title'].extend(papername)
    paper_repos_dict['Year'].extend(year)
    paper_repos_dict['Author'].extend(author)
    #paper_repos_dict['Citation'].extend(cite)
    paper_repos_dict['Publication'].extend(publi)
    paper_repos_dict['Url of paper'].extend(link)

    return pd.DataFrame(paper_repos_dict)

In [11]:
from time import sleep
#last_page_num = get_page_num(paper, 10) #get last page
#Scrape Multiple Pages
for i in range(0,20,10):
    # get url for the each page
    url = "https://scholar.google.com/scholar?start={}&hl=en&as_sdt=2005&sciodt=2006&cites=11660772839861173263&scipsc=".format(i)
    # function for the get content of each page
    doc = get_paperinfo(url)

    # function for the collecting tags
    paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
    
    # paper title from each page
    paper_title = get_papertitle(paper_tag)
    
    # year , author , publication of the paper
    year , publication , author = get_author_year_publi_info(author_tag)
    
    # cite count of the paper 
     #cite = get_citecount(cite_tag)

    # url of the paper
    link = get_link(link_tag)
    
    #get iteration:
    print(i)
    print("----------------------------------------------")
    # add in paper repo dict
    final = add_in_paper_repo(paper_title,year,author,publication,link)
    rint = random.randint(30, 90)
    print(rint) #shows rint
    # use sleep to avoid status code 429
    sleep(rint)

0
----------------------------------------------


NameError: name 'random' is not defined

In [99]:
final

,Paper Title,Year,Author,Publication,Url of paper
0,[HTML][HTML] Environmental neuroeconomics: how...,2021,N Sawe,Elsevier,https://www.sciencedirect.com/science/article/...
1,Decision neuroscience and neuroeconomics: Rece...,2022,JB Dennison,Library,https://wires.onlinelibrary.wiley.com/doi/abs/...
2,[HTML][HTML] Data Triangulation in Consumer Ne...,2020,CC Cao,frontiersin.org,https://www.frontiersin.org/articles/10.3389/f...
3,Neuromarketing: How to Choose the Right Measures,2020,S Bellman,taylorfrancis.com,https://www.taylorfrancis.com/chapters/edit/10...
4,Neuroscience in service research: an overview ...,2019,N Verhulst,emerald.com,https://www.emerald.com/insight/content/doi/10...
5,[HTML][HTML] Media content sharing as a value-...,2020,C Scholz,Elsevier,https://www.sciencedirect.com/science/article/...
6,Using neuroimaging to predict behavior: An ove...,2019,SH Tompson,psycnet.apa.org,https://psycnet.apa.org/record/2019-55953-010
7,Studying volition with actions that matter: Co...,2020,L Mudrik,psycnet.apa.org,https://psycnet.apa.org/buy/2019-76272-001
8,Neuroscience in service research: an overview ...,2020,A De,biopen.bi.no,https://biopen.bi.no/bi-xmlui/handle/11250/263...
9,"[HTML][HTML] Representation, pattern informati...",2018,PA Kragel,Elsevier,https://www.sciencedirect.com/science/article/...


In [53]:
#send to csv
final.to_csv('aggregate_demand_neuroforecasting10.csv', sep=',', index=False,header=True)
